Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [3]:
labels = [{'name':'Hello', 'id':1}, {'name':'yes', 'id':2},
         {'name':'no', 'id':3}, {'name':'thank you', 'id':4},
         {'name':'i love you', 'id':5}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [7]:
!set PYTHONPATH=C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\models\research;C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\models\research\slim;%PYTHONPATH% && python "{SCRIPTS_PATH}/generate_tfrecord.py" -x "{IMAGE_PATH}/train" -l "{ANNOTATION_PATH}/label_map.pbtxt" -o "{ANNOTATION_PATH}/train.record"

!set PYTHONPATH=C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\models\research;C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\models\research\slim;%PYTHONPATH% && python "{SCRIPTS_PATH}/generate_tfrecord.py" -x "{IMAGE_PATH}/test" -l "{ANNOTATION_PATH}/label_map.pbtxt" -o "{ANNOTATION_PATH}/test.record"


Successfully created the TFRecord file: C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\workspace\annotations/train.record




W1031 11:21:48.977504  7176 module_wrapper.py:149] From C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\scripts/generate_tfrecord.py:153: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W1031 11:21:49.421239  7176 module_wrapper.py:149] From C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\scripts/generate_tfrecord.py:111: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



Successfully created the TFRecord file: C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\workspace\annotations/test.record




W1031 11:22:00.397893 15048 module_wrapper.py:149] From C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\scripts/generate_tfrecord.py:153: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W1031 11:22:00.608653 15048 module_wrapper.py:149] From C:\Users\HP\Documents\RealTimeObjectDetection\Tensorflow\scripts/generate_tfrecord.py:111: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [5]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  38% (1519/3959)
Updating files:  39% (1545/3959)
Updating files:  40% (1584/3959)
Updating files:  41% (1624/3959)
Updating files:  42% (1663/3959)
Updating files:  43% (1703/3959)
Updating files:  44% (1742/3959)
Updating files:  45% (1782/3959)
Updating files:  46% (1822/3959)
Updating files:  47% (1861/3959)
Updating files:  48% (1901/3959)
Updating files:  49% (1940/3959)
Updating files:  50% (1980/3959)
Updating files:  51% (2020/3959)
Updating files:  52% (2059/3959)
Updating files:  53% (2099/3959)
Updating files:  54% (2138/3959)
Updating files:  55% (2178/3959)
Updating files:  56% (2218/3959)
Updating files:  57% (2257/3959)
Updating files:  58% (2297/3959)
Updating files:  59% (2336/3959)
Updating files:  60% (2376/3959)
Updating files:  61% (2415/3959)
Updating files:  62% (2455/3959)
Updating files:  63% (2495/3959)
Updating files:  64% (2534/3959)
Updating files:  65% (2574/3959)
Updating files:  65% (2590/3959)
Updating files:  6

In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [8]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [10]:
import os
import shutil

CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_PATH = os.path.join('Tensorflow', 'workspace', 'pre-trained-models', 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8')
MODEL_PATH = os.path.join('Tensorflow', 'workspace', 'models')
CUSTOM_MODEL_PATH = os.path.join(MODEL_PATH, CUSTOM_MODEL_NAME)

# create model folder if it doesn't exist
os.makedirs(CUSTOM_MODEL_PATH, exist_ok=True)

# copy pipeline.config file
shutil.copy(os.path.join(PRETRAINED_MODEL_PATH, 'pipeline.config'),
            os.path.join(CUSTOM_MODEL_PATH, 'pipeline.config'))

print("✅ Folder ready at:", CUSTOM_MODEL_PATH)
print("✅ pipeline.config copied successfully!")


✅ Folder ready at: Tensorflow\workspace\models\my_ssd_mobnet
✅ pipeline.config copied successfully!


# 5. Update Config For Transfer Learning

In [5]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [6]:
import sys
print(sys.executable)



C:\Users\HP\anaconda3\envs\tf2_od\python.exe


In [2]:
!where python


C:\Users\HP\anaconda3\python.exe
C:\Users\HP\AppData\Local\Microsoft\WindowsApps\python.exe


In [11]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [12]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

config

In [14]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [15]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [16]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [17]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [19]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow\workspace\models/my_ssd_mobnet --pipeline_config_path=Tensorflow\workspace\models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 7. Load Train Model From Checkpoint

In [3]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [6]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-15')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [7]:
import cv2 
import numpy as np

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [10]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [ ]:
detections = detect_fn(input_tensor)

In [ ]:
from matplotlib import pyplot as plt